In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

## **Loading Dataset**

In [50]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

In [51]:
cols = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
        "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

In [52]:
data = pd.read_csv(url, names=cols)

In [53]:
x = data[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
        "BMI", "DiabetesPedigreeFunction", "Age"]].values

y = data[['Outcome']].values

## **Data Normalization**

In [54]:
x = (x - x.min())/(x.max() - x.min())

## **Separating the Dataset**

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=27)

## **Creating the Dataset Object**

In [56]:
class SpecificDataset(Dataset):
  def __init__(self, x, y):
    self.features = torch.tensor(x, dtype=torch.float32)
    self.labels = torch.tensor(y, dtype=torch.float32)

  def __getitem__(self, index):
    x = self.features[index]
    y = self.labels[index]

    return x, y

  def __len__(self):
    return self.labels.shape[0]

In [57]:
train_dataset = SpecificDataset(x_train, y_train)
test_dataset = SpecificDataset(x_test, y_test)

In [58]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=10,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=10,
                             shuffle=True)

## **Perceptron**

In [59]:
class Perceptron(torch.nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = torch.nn.Linear(num_features, 1)

  def forward(self, x):
    z = self.linear(x)

    return torch.sigmoid(z)

In [60]:
torch.manual_seed(27)

In [61]:
model = Perceptron(8)

In [62]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.0)

## **Training**

In [ ]:
epochs = 10

In [63]:
for epoch in range(epochs):
  model = model.train()

  for index, (x, y) in enumerate(train_dataloader):
    a = model(x)
    loss = F.binary_cross_entropy(a, y.view(a.shape))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch + 1:03d}/{epochs:03d} | Batch: {index + 1:03d}/{len(train_dataloader):03d} | Loss: {loss:.3f}')

Epoch: 001/010 | Batch: 001/054 | Loss: 0.675
Epoch: 001/010 | Batch: 002/054 | Loss: 0.644
Epoch: 001/010 | Batch: 003/054 | Loss: 0.682
Epoch: 001/010 | Batch: 004/054 | Loss: 0.675
Epoch: 001/010 | Batch: 005/054 | Loss: 0.629
Epoch: 001/010 | Batch: 006/054 | Loss: 0.729
Epoch: 001/010 | Batch: 007/054 | Loss: 0.681
Epoch: 001/010 | Batch: 008/054 | Loss: 0.683
Epoch: 001/010 | Batch: 009/054 | Loss: 0.677
Epoch: 001/010 | Batch: 010/054 | Loss: 0.716
Epoch: 001/010 | Batch: 011/054 | Loss: 0.619
Epoch: 001/010 | Batch: 012/054 | Loss: 0.732
Epoch: 001/010 | Batch: 013/054 | Loss: 0.599
Epoch: 001/010 | Batch: 014/054 | Loss: 0.678
Epoch: 001/010 | Batch: 015/054 | Loss: 0.621
Epoch: 001/010 | Batch: 016/054 | Loss: 0.618
Epoch: 001/010 | Batch: 017/054 | Loss: 0.748
Epoch: 001/010 | Batch: 018/054 | Loss: 0.627
Epoch: 001/010 | Batch: 019/054 | Loss: 0.621
Epoch: 001/010 | Batch: 020/054 | Loss: 0.743
Epoch: 001/010 | Batch: 021/054 | Loss: 0.675
Epoch: 001/010 | Batch: 022/054 | 

## **Testing**

In [64]:
def accuracy(model, dataloader):
  model = model.eval()

  accuracy = 0.0
  total = 0
  for index, (x, y) in enumerate(dataloader):

    with torch.inference_mode():
      z = model(x)

    a = torch.where(z > 0.5, 1, 0)
    labels = y.view(a.shape).to(a.dtype)

    result = a == labels
    accuracy += torch.sum(result)
    total += len(result)

  return (accuracy / total).item()

In [65]:
accuracy(model, test_dataloader)

0.6580086350440979